### Requirements

In [1]:
pip install --upgrade huggingface_hub

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.4 MB/s eta 0:00:00


In [4]:
pip install transformers

In [5]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:00


In [6]:
!pip3 install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.6 MB/s eta 0:00:00a 0:00:01


### Import

In [7]:
import pandas as pd

In [8]:
import torch

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

### Loading Reference Data

In [11]:
df_one = pd.read_csv("KnowledgeTransfers-one_sent.csv")

In [12]:
### Selecting all annotated cases of family relations
df_fam_one = df_one[df_one.relation.isin(["parent_of", "child_of", "siblings", "spouses"])].copy(deep=True)
df_fam_one.reset_index(drop=True, inplace=True)

In [13]:
df_fam_one.relation.value_counts()

parent_of    29
child_of     26
siblings     23
spouses      11
Name: relation, dtype: int64

In [14]:
df_fam_one.shape

(89, 11)

In [15]:
df_fam_one.drop([11, 19, 20, 22], inplace=True)
df_fam_one.reset_index(drop=True, inplace=True)

### Loading Models

In [16]:
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  #bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type = "nf4",
  bnb_4bit_compute_dtype = torch.float16
)

In [17]:
model_id_1 = "meta-llama/Llama-2-7b-chat-hf"

tokenizer_1 = AutoTokenizer.from_pretrained(model_id_1, use_auth_token=True, use_fast=True)
model_7b = AutoModelForCausalLM.from_pretrained(model_id_1, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

hf_7b = pipeline("text-generation", model=model_7b, tokenizer=tokenizer_1, temperature = 0.001)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [18]:
model_id_2 = "meta-llama/Llama-2-13b-chat-hf"

tokenizer_2 = AutoTokenizer.from_pretrained(model_id_2, use_auth_token=True, use_fast=True)
model_13b = AutoModelForCausalLM.from_pretrained(model_id_2, use_auth_token=True, device_map="auto", quantization_config=quantization_config)

hf_13b = pipeline("text-generation", model=model_13b, tokenizer=tokenizer_2, temperature = 0.001)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Creating Prompt Templates

In [79]:
def generate(prompt, max_length=1024, pipe=hf_13b, **kwargs):
    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

In [80]:
def prompt_with_examples(prompt, examples=[]):

    # Start with the initial part of the prompt
    full_prompt = "<s>[INST]\n"

    # Add each example to the prompt
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

#### Building Examples for Few-Shot Prompts

In [81]:
example_parent = "LUCIE einen Mantelsack tragend, zu Karl. Laß Er's nur, es ist nicht schwer; aber nehm Er meiner Mutter die Schachtel ab. POSTMEISTERIN. Ihre Dienerin, meine Frauenzimmer!"

In [82]:
example_child = "Lucie tritt auf. FERNANDO. O meine Tochter! LUCIE. Lieber, bester Vater!"

In [83]:
example_sibling = "Deine Frau? Hoffmann. Nein, ihre Schwester. Hörtest Du nicht, wie sie mich betitelte? Loth."

In [84]:
example_spouses = "Es wird so Nacht! FERNANDO. Und dieses Weib ist meine Frau! – STELLA"

In [85]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_parent}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_parent = "parent_of"
example_parent = (example_prompt, example_response_parent)

In [86]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_child}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_child = "child_of"
example_child = (example_prompt, example_response_child)

In [87]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_sibling}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_sibling = "siblings"
example_sibling = (example_prompt, example_response_sibling)

In [88]:
example_prompt = f"""
What kind of family relationship is conveyed in the following German {example_spouses}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_spouses = "spouses"
example_spouses = (example_prompt, example_response_spouses)

In [89]:
examples = [
    example_parent,
    example_child,
    example_sibling,
    example_spouses
    ]

### Runing Prompts on One-Sentences-DF





In [90]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [91]:
###Zero Shot
results = []
for text in df_fam_one.context:
  drama_snippet = text

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", "child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [92]:
results_1 = []
for text in results:
  if "parent_of" in text:
    results_1.append("parent_of")
  elif "child_of" in text:
    results_1.append("child_of")
  elif "siblings" in text:
    results_1.append("siblings")
  elif "spouses" in text:
    results_1.append("spouses")
  elif "Parent_of" in text:
    results_1.append("parent_of")
  elif "Child_of" in text:
    results_1.append("child_of")
  elif "Siblings" in text:
    results_1.append("siblings")
  elif "Spouses" in text:
    results_1.append("spouses")
  else:
    results_1.append("NaN")

In [93]:
df_fam_one_zero = df_fam_one.copy(deep=True)

In [94]:
results_se = pd.Series(results_1)

In [95]:
df_fam_one_zero.famrel_pred = results_se

<ipython-input-95-cc594ed24f7e>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_one_zero.famrel_pred = results_se


In [96]:
ground_truth = df_fam_one_zero.relation
predictions = df_fam_one_zero.famrel_pred

In [97]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.477861,0.505882,0.59862,0.505882


In [98]:
###Zero Shot 2
results_0 = []
for text in df_fam_one.context:
  drama_snippet = text

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of the following labels:
  A: "child_of"
  B: "parent_of"
  C: "siblings"
  D: "spouses".
  JUST name the label, do NOT generate any more text!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results_0.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [99]:
results_00 = []
for text in results_0:
  if "parent_of" in text:
    results_00.append("parent_of")
  elif "child_of" in text:
    results_00.append("child_of")
  elif "siblings" in text:
    results_00.append("siblings")
  elif "spouses" in text:
    results_00.append("spouses")
  elif "Parent_of" in text:
    results_00.append("parent_of")
  elif "Child_of" in text:
    results_00.append("child_of")
  elif "Siblings" in text:
    results_00.append("siblings")
  elif "Spouses" in text:
    results_00.append("spouses")
  else:
    results_00.append("NaN")

In [100]:
df_fam_one_zero_2 = df_fam_one.copy(deep=True)

In [101]:
len(results_00)

85

In [102]:
results_se_0 = pd.Series(results_00)

In [103]:
df_fam_one_zero_2.famrel_pred = results_se_0

<ipython-input-103-21ec9426dc63>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_one_zero_2.famrel_pred = results_se_0


In [104]:
ground_truth = df_fam_one_zero_2.relation
predictions = df_fam_one_zero_2.famrel_pred

In [105]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.555876,0.564706,0.558074,0.564706


In [43]:
examples = [
    example_parent,
    example_child,
    example_sibling,
    example_spouses
    ]

In [106]:
###Few-Shot
results_2 = []
for text in df_fam_one.context:
  drama_snippet = text

  prompt = f"""
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  """

  prompt_exam = prompt_with_examples(prompt, examples)

  result = generate(prompt_exam)
  results_2.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
###Few-Shot 2
results_2 = []
for text in df_fam_one.context:
  drama_snippet = text

  prompt = f"""
  What kind of family relationship is conveyed in the following German {drama_snippet}?

  Choose one of the following labels:
  A: "child_of"
  B: "parent_of"
  C: "siblings"
  D: "spouses".
  JUST name the label, do NOT generate any more text!
  Family relation:
  [/INST]
  """

  prompt_exam = prompt_with_examples(prompt, examples)

  result = generate(prompt_exam)
  results_2.append(result)

In [107]:
df_fam_one_few = df_fam_one.copy(deep=True)

In [108]:
results_2_2 = []
for text in results_2:
  if "parent_of" in text:
    results_2_2.append("parent_of")
  elif "child_of" in text:
    results_2_2.append("child_of")
  elif "siblings" in text:
    results_2_2.append("siblings")
  elif "spouses" in text:
    results_2_2.append("spouses")
  elif "Parent_of" in text:
    results_2_2.append("parent_of")
  elif "Child_of" in text:
    results_2_2.append("child_of")
  elif "Siblings" in text:
    results_2_2.append("siblings")
  elif "Spouses" in text:
    results_2_2.append("spouses")
  else:
    results_2_2.append("NaN")

In [109]:
results_2_se = pd.Series(results_2_2)

In [110]:
results_2_se.value_counts()

parent_of    37
siblings     21
spouses      14
NaN           7
child_of      6
dtype: int64

In [111]:
df_fam_one_few.famrel_pred = results_2_se

<ipython-input-111-704b47529bc8>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_one_few.famrel_pred = results_2_se


In [112]:
df_fam_one_few.famrel_pred = df_fam_one_few.famrel_pred.replace('mother', 'parent_of')
df_fam_one_few.famrel_pred = df_fam_one_few.famrel_pred.replace('daughter', 'child_of')

In [113]:
ground_truth_few = df_fam_one_few.relation
predictions_few = df_fam_one_few.famrel_pred

In [114]:
pd.DataFrame(list(zip([f1_score(ground_truth_few, predictions_few, average="weighted")],
                      [recall_score(ground_truth_few, predictions_few,  average="weighted")],
                      [precision_score(ground_truth_few, predictions_few, average="weighted")],
                      [accuracy_score(ground_truth_few, predictions_few,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.408175,0.435294,0.413937,0.435294


In [115]:
###Zero Shot with Names
results_3 = []
for index, row in df_fam_one.iterrows():
  person_1 = row["arg1"]
  person_2 = row["arg2"]
  drama_snippet = row["context"]

  prompt = f"""
  <s>[INST]\n
  What kind of family relationship between {person_1} and {person_2} is conveyed in the following German {drama_snippet}?

  Choose one of the following labels:
  A: "child_of"
  B: "parent_of"
  C: "siblings"
  D: "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  result = generate(prompt)
  results_3.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [116]:
results_4 = []
for text in results_3:
  if "parent_of" in text:
    results_4.append("parent_of")
  elif "child_of" in text:
    results_4.append("child_of")
  elif "siblings" in text:
    results_4.append("siblings")
  elif "spouses" in text:
    results_4.append("spouses")
  elif "Parent_of" in text:
    results_4.append("parent_of")
  elif "Child_of" in text:
    results_4.append("child_of")
  elif "Siblings" in text:
    results_4.append("siblings")
  elif "Spouses" in text:
    results_4.append("spouses")
  else:
    results_4.append("NaN")

In [117]:
df_fam_one_pers_zero = df_fam_one.copy(deep=True)

In [118]:
results_se_3 = pd.Series(results_4)

In [119]:
df_fam_one_pers_zero.famrel_pred = results_se_3

<ipython-input-119-7f3a9606b9ba>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_one_pers_zero.famrel_pred = results_se_3


In [120]:
ground_truth_3 = df_fam_one_pers_zero.relation
predictions_3 = df_fam_one_pers_zero.famrel_pred

In [121]:
pd.DataFrame(list(zip([f1_score(ground_truth_3, predictions_3, average="weighted")],
                      [recall_score(ground_truth_3, predictions_3,  average="weighted")],
                      [precision_score(ground_truth_3, predictions_3, average="weighted")],
                      [accuracy_score(ground_truth_3, predictions_3)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.663693,0.682353,0.686327,0.682353


In [ ]:
###Few Shot with Names

In [122]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Madame Sommer und Lucie in the following German {example_parent}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_parent = "parent_of"
example_parent_pers = (example_prompt_pers, example_response_parent)

In [123]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Lucie and Fernando in the following German {example_child}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_child = "child_of"
example_child_pers = (example_prompt_pers, example_response_child)

In [124]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Fernando and Madame Sommer in the following German {example_sibling}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_sibling = "siblings"
example_sibling_pers = (example_prompt_pers, example_response_sibling)

In [125]:
example_prompt_pers = f"""
What kind of family relationship is conveyed between Helene and the NaN in the following German {example_spouses}?

Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label and nothing else!
Family relation:
"""
example_response_spouses = "spouses"
example_spouses_pers = (example_prompt_pers, example_response_spouses)

In [126]:
examples_pers = [
    example_parent_pers,
    example_child_pers,
    example_sibling_pers,
    example_spouses_pers
    ]

In [127]:
###Few-Shot with Names
results_5 = []
for index, row in df_fam_one.iterrows():
  person_1 = row["arg1"]
  person_2 = row["arg2"]
  drama_snippet = row["context"]

  prompt_pers = f"""
  <s>[INST]\n
  What kind of family relationship between {person_1} and {person_2} is conveyed in the following German {drama_snippet}?

  Choose one of "parent_of", child_of", "siblings", "spouses".
  JUST name the label and nothing else!
  Family relation:
  [/INST]
  """

  prompt_exam_pers = prompt_with_examples(prompt_pers, examples_pers)

  result = generate(prompt_exam_pers)
  results_5.append(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1182, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [128]:
df_fam_one_pers_few = df_fam_one.copy(deep=True)

In [129]:
results_4_se = pd.Series(results_5)

In [130]:
results_4_se

0     parent_of
1       spouses
2     parent_of
3      siblings
4      siblings
        ...    
80     siblings
81      spouses
82      spouses
83    parent_of
84     child_of
Length: 85, dtype: object

In [131]:
df_fam_one_pers_few.famrel_pred = results_4_se

<ipython-input-131-9654a83c1752>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fam_one_pers_few.famrel_pred = results_4_se


In [132]:
ground_truth_4 = df_fam_one_pers_few.relation
predictions_4 = df_fam_one_pers_few.famrel_pred

In [133]:
pd.DataFrame(list(zip([f1_score(ground_truth_4, predictions_4, average="weighted")],
                      [recall_score(ground_truth_4, predictions_4,  average="weighted")],
                      [precision_score(ground_truth_4, predictions_4, average="weighted")],
                      [accuracy_score(ground_truth_4, predictions_4)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.631153,0.635294,0.714605,0.635294
